In [69]:
### Leaky Bukcet
import asyncio
import queue
import time

class LeakyBucket:
    """This class implements the leaky bucket"""
    def __init__(self,max_rate=1,capacity=10):
        self.max_rate = max_rate # sec to wait for the requests to process
        self.capacity = capacity
        self._current_capacity = 0
        self._prev_time = time.time()
        self._leak_queue = []
        self._id=0
        self._leak=None
    
    def _leak_manager(self):
        while len(self._leak_queue)>0:
            amt = self._leak_queue[0]
            if (amt[0]+self._current_capacity)<=self.capacity:
                print(f"in _leak_manager, processing the {amt[1]}")
                self._current_capacity = amt[0]+self._current_capacity
                self._leak_queue.pop(0)
            else:
                yield
        self._leak = None

    def call_next(self):
        if self._leak is not None:
            next(self._leak)
    
    def add_next(self,amount):    
        self._leak_queue.append(amount)
        self._leak = self._leak_manager()
    def fill_bucket(self,n_requests=1):
        """This is the implementation for the filling the bucket"""
        self.call_next()
        start = time.time()
        elapsed = self._prev_time-start
        requests_to_be_dropped = int(self.max_rate*elapsed)
        self._current_capacity = max([0,self._current_capacity-requests_to_be_dropped])
        self._prev_time = start
        if n_requests+self._current_capacity>self.capacity:
            self.add_next([n_requests,self._id])
        else:
            print(f"in fill_bucket, processing the {self._id}")
            self._current_capacity=n_requests+self._current_capacity
        self._id+=1
        

In [70]:
bucket=LeakyBucket(capacity=40,max_rate=10)

In [ ]:
bucket.fill_bucket(10)
bucket.fill_bucket(50)
bucket.fill_bucket(20)
bucket.fill_bucket(30)
bucket.fill_bucket(40)

In [27]:
# Edit distance algo

del_cost=1
ins_cost=1
update_cost = 2

def edit_distance(s1,s2):
    """
    This algo approaches the dynamic problem by solving the sub problems to reach the final optimal solution
     Making s1 to s2
     where algo moves by each row and makes the decision at each row level
     last row will give the maximum dp cost
    """

    n= len(s1)
    m = len(s2)
    dp = []
    for i in range(n+1):
        c=[]
        for j in range(m+1):
            c.append([0,0].copy())
        dp.append(c.copy())
    for i in range(1,n+1):
        dp[i][0][0] = dp[i-1][0][0]+del_cost
    for i in range(1,m+1):
        dp[0][i][0] = dp[0][i-1][0]+ins_cost
    for i in range(1,n+1):
        for j in range(1,m+1):
            cost_idx=[[i-1,j],[i,j-1],[i-1,j-1]]
            cost = [dp[i-1][j][0]+del_cost,dp[i][j-1][0]+ins_cost,dp[i-1][j-1][0]+0 if s2[i-1]==s1[j-1] else dp[i-1][j-1][0]+update_cost]
            dp[i][j][0] = min(cost)
            dp[i][j][1] = cost_idx[cost.index(min(cost))]
    print(dp[n][m][0])
    end = dp[n][m][1]
    letter = [(s1[n-1],s2[m-1])]
    while True:
        end_ = dp[end[0]][end[1]][1]
        letter.append((s1[end[0]],s2[end[0]]))
        end=end_
        if not isinstance(end_,list):
            break
    print(dp)
    print(s1,s2)
    print(letter[::-1])
edit_distance("intention","execution")
edit_distance("leda","deal")

8
[[[0, 0], [1, 0], [2, 0], [3, 0], [4, 0], [5, 0], [6, 0], [7, 0], [8, 0], [9, 0]], [[1, 0], [2, [0, 1]], [3, [0, 2]], [4, [0, 3]], [3, [0, 3]], [4, [1, 4]], [5, [1, 5]], [6, [1, 6]], [7, [1, 7]], [8, [1, 8]]], [[2, 0], [3, [1, 1]], [4, [1, 2]], [5, [1, 3]], [4, [1, 4]], [5, [1, 5]], [6, [1, 6]], [7, [1, 7]], [8, [1, 8]], [9, [1, 9]]], [[3, 0], [4, [2, 1]], [5, [2, 2]], [6, [2, 3]], [5, [2, 4]], [6, [2, 5]], [7, [2, 6]], [8, [2, 7]], [9, [2, 8]], [10, [2, 9]]], [[4, 0], [5, [3, 1]], [6, [3, 2]], [7, [3, 3]], [6, [3, 4]], [7, [3, 5]], [8, [3, 6]], [9, [3, 7]], [10, [3, 8]], [11, [3, 9]]], [[5, 0], [6, [4, 1]], [7, [4, 2]], [8, [4, 3]], [7, [4, 4]], [8, [4, 5]], [9, [4, 6]], [10, [4, 7]], [11, [4, 8]], [12, [4, 9]]], [[6, 0], [7, [5, 1]], [8, [5, 2]], [7, [5, 2]], [8, [5, 4]], [9, [5, 5]], [8, [5, 5]], [9, [6, 6]], [10, [6, 7]], [11, [6, 8]]], [[7, 0], [6, [6, 0]], [7, [7, 1]], [8, [6, 3]], [9, [6, 4]], [10, [6, 5]], [9, [6, 6]], [8, [6, 6]], [9, [7, 7]], [10, [7, 8]]], [[8, 0], [7, [7,